## Optimizing an Apache MXNet model for AWS DeepLens

This notebook shows you how to use the Intel Deep Learning Deployment Toolkit to optimize an MXNet model for Deep Lens.

In [16]:
%env TOOLKIT_BUCKET=s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/
%env TOOLKIT_NAME=toolkit.tgz
%env TOOLKIT_DIR=l_deeplearning_deploymenttoolkit_2017.1.0.5852

%env MODEL_BUCKET=s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/model/gender/sagemaker-gender-training-notebook/output/sagemaker-gender-training-notebook-2018-01-23-10-54-15/output/
%env MODEL_TAR_BALL_NAME=model.tar.gz
%env MODEL_NAME=image-classification
%env MODEL_EPOCH=0060
%env MODEL_DIR=model/

%env OPT_DIR=/opt/intel/deeplearning_deploymenttoolkit/deployment_tools/model_optimizer/model_optimizer_mxnet
%env OPT_PRECISION=FP16
%env OPT_FUSE=YES

env: TOOLKIT_BUCKET=s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/
env: TOOLKIT_NAME=toolkit.tgz
env: TOOLKIT_DIR=l_deeplearning_deploymenttoolkit_2017.1.0.5852
env: MODEL_BUCKET=s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/model/gender/sagemaker-gender-training-notebook/output/sagemaker-gender-training-notebook-2018-01-23-10-54-15/output/
env: MODEL_TAR_BALL_NAME=model.tar.gz
env: MODEL_NAME=image-classification
env: MODEL_EPOCH=0060
env: MODEL_DIR=model/
env: OPT_DIR=/opt/intel/deeplearning_deploymenttoolkit/deployment_tools/model_optimizer/model_optimizer_mxnet
env: OPT_PRECISION=FP16
env: OPT_FUSE=YES


In [17]:
%%bash

echo "*** Downloading toolkit"
aws s3 cp $TOOLKIT_BUCKET$TOOLKIT_NAME .
echo "*** Installing toolkit"
tar xfz $TOOLKIT_NAME
cd $TOOLKIT_DIR
chmod 755 install.sh
sudo ./install.sh -s silent.cfg 
echo "*** Done"

*** Downloading toolkit
download: s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/toolkit.tgz to ./toolkit.tgz
*** Installing toolkit
--------------------------------------------------------------------------------
The install directory path was changed to
/opt/intel/deeplearning_deploymenttoolkit_2017.1.0.5852
because at least one software product component was detected as having already
been installed on the system.
The Intel's Deep Learning Deployment Toolkit is already installed.

If you want to reinstall the Intel's Deep Learning Deployment Toolkit please
uninstall current version and run install script again.
--------------------------------------------------------------------------------
*** Done


In [18]:
%%bash

#conda create -n intel_toolkit -y
python -m ipykernel install --user --name intel_toolkit --display-name "intel_toolkit"

source activate intel_toolkit
cd $OPT_DIR
pip install -r requirements.txt 

pip install mxnet==1.0.0

Installed kernelspec intel_toolkit in /home/ec2-user/.local/share/jupyter/kernels/intel_toolkit
  Using cached mxnet-0.11.0-py2.py3-none-manylinux1_x86_64.whl
  Found existing installation: mxnet 1.0.0
    Uninstalling mxnet-1.0.0:
      Successfully uninstalled mxnet-1.0.0
  Using cached mxnet-1.0.0-py2.py3-none-manylinux1_x86_64.whl
  Found existing installation: mxnet 0.11.0
    Uninstalling mxnet-0.11.0:
      Successfully uninstalled mxnet-0.11.0


In [19]:
%%bash

echo 'Download model'
aws s3 cp $MODEL_BUCKET$MODEL_TAR_BALL_NAME $MODEL_DIR

echo 'Extract model files'
cd $MODEL_DIR
tar xzvf $MODEL_TAR_BALL_NAME $MODEL_NAME'-'$MODEL_EPOCH'.params' $MODEL_NAME'-symbol.json' 'model-shapes.json'

echo 'Rename model parameter file'
mv $MODEL_NAME'-'$MODEL_EPOCH'.params' $MODEL_NAME'-0000.params'
ls -al

Download model
download: s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/model/gender/sagemaker-gender-training-notebook/output/sagemaker-gender-training-notebook-2018-01-23-10-54-15/output/model.tar.gz to model/model.tar.gz
Extract model files
image-classification-symbol.json
image-classification-0060.params
model-shapes.json
Rename model parameter file
total 2102296
drwxrwxr-x 2 ec2-user ec2-user       4096 Feb  9 04:42 .
drwxr-xr-x 7 ec2-user ec2-user       4096 Feb  9 04:41 ..
-rw-r--r-- 1 ec2-user ec2-user  170415851 Jan 23 22:27 image-classification-0000.params
-rw-r--r-- 1 ec2-user ec2-user     215316 Jan 23 22:27 image-classification-symbol.json
-rw-r--r-- 1 ec2-user ec2-user         46 Jan 23 11:01 model-shapes.json
-rw-rw-r-- 1 ec2-user ec2-user 1896956015 Jan 23 22:38 model.tar.gz
-rw-rw-r-- 1 ec2-user ec2-user   84960784 Feb  9 04:25 mxnet_image-classification_FP16_FUSED.bin
-rw-rw-r-- 1 ec2-user ec2-user     173061 Feb  9 04:25 mxnet_image-classification_FP16_

## Workarounds

SageMaker and DeepLens has some known model compatibility issue, here are workarounds suggested by AWS employee on DeepLens forum post [here](https://forums.aws.amazon.com/thread.jspa?threadID=270792&tstart=0). There are many forum posts about model optimisation issues. If you see anything new, please write a forum post someone from AWS will help you there.

Further more, DeepLens module awscam v1.2.1 has a new python library for model optimisation called "mo", please use that library instead if you have awscam v1.2.1 installed on DeepLens device.

In [20]:
import os
import json

model_name = os.environ['MODEL_NAME']
model_path = os.environ['MODEL_DIR']

symbol_name = model_name + '-symbol.json'
symbol_path = model_path + '/' + symbol_name
symbol_json = json.load(open(symbol_path, 'r'))
nodes = symbol_json['nodes']
print('Applying workarounds to model symbol file: ' + symbol_name)
print('Ref: https://forums.aws.amazon.com/thread.jspa?threadID=270792&tstart=0')
for node in nodes:
    if ('op' in node ) and (node['op'] != 'null') and ('name' in node) and (node['name'].endswith('_fwd')):
        print('Workaround: Striping suffix \"_fwd\" in layer name of non-null operators')
        print('Original')
        print(node)
        node['name'] = node['name'][:-4]
        print('Modified')
        print(node)
    elif ('op' in node) and (node['op'] == 'Pooling') and ('attr' in node) and ('stride' not in node['attr']):
        print('Workaround: Pooling layer missing \"stride\" in \"attr\"')
        print('Original')
        print(node)
        node['attr']['stride'] = '(1, 1)'
        print('Modified')
        print(node)
    elif ('op' in node) and (node['op'] == 'SoftmaxOutput') and ('attr' not in node):
        print('Workaround: SoftmaxOutput layer missing \"attr\"')
        print('Original')
        print(node)
        node['attr'] = {}
        print('Modified')
        print(node)

json.dump(symbol_json, open(symbol_path, 'w'))

Applying workarounds to model symbol file: image-classification-symbol.json
Ref: https://forums.aws.amazon.com/thread.jspa?threadID=270792&tstart=0
Workaround: Pooling layer missing "stride" in "attr"
Original
{'op': 'Pooling', 'name': 'pool1', 'attr': {'global_pool': 'True', 'kernel': '(7, 7)', 'pool_type': 'avg'}, 'inputs': [[853, 0, 0]]}
Modified
{'op': 'Pooling', 'name': 'pool1', 'attr': {'global_pool': 'True', 'kernel': '(7, 7)', 'pool_type': 'avg', 'stride': '(1, 1)'}, 'inputs': [[853, 0, 0]]}
Workaround: SoftmaxOutput layer missing "attr"
Original
{'op': 'SoftmaxOutput', 'name': 'softmax', 'inputs': [[858, 0, 0], [859, 0, 0]]}
Modified
{'op': 'SoftmaxOutput', 'name': 'softmax', 'inputs': [[858, 0, 0], [859, 0, 0]], 'attr': {}}


## Model Optimisation

In [21]:
%%bash

source activate intel_toolkit

echo "*** Converting model"
python3 $OPT_DIR/mo_mxnet_converter.py --models-dir $MODEL_DIR --output-dir $MODEL_DIR --model-name $MODEL_NAME --precision $OPT_PRECISION --fuse $OPT_FUSE

cd $MODEL_DIR
ls .
aws s3 cp mxnet_image-classification_FP16_FUSED.bin $MODEL_BUCKET
aws s3 cp mxnet_image-classification_FP16_FUSED.xml $MODEL_BUCKET

echo "*** Done"

*** Converting model
image-classification-0000.params
image-classification-symbol.json
model-shapes.json
model.tar.gz
mxnet_image-classification_FP16_FUSED.bin
mxnet_image-classification_FP16_FUSED.xml
upload: ./mxnet_image-classification_FP16_FUSED.bin to s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/model/gender/sagemaker-gender-training-notebook/output/sagemaker-gender-training-notebook-2018-01-23-10-54-15/output/mxnet_image-classification_FP16_FUSED.bin
upload: ./mxnet_image-classification_FP16_FUSED.xml to s3://deeplens-sagemaker-2bbe16b4-c056-4ae2-9332-d31dd7aeb470/model/gender/sagemaker-gender-training-notebook/output/sagemaker-gender-training-notebook-2018-01-23-10-54-15/output/mxnet_image-classification_FP16_FUSED.xml
*** Done


[04:46:27] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v0.12.0. Attempting to upgrade...
[04:46:27] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
/home/ec2-user/anaconda3/envs/intel_toolkit/lib/python3.6/site-packages/numba/serialize.py:7: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [ ]:
%%bash

echo 'Cleaning'

echo 'Remove model folder'
rm -rf $MODEL_DIR
rm -rf $TOOLKIT_DIR

echo 'Remove downloaded files'
rm $TOOLKIT_NAME

source activate intel_toolkit

echo 'Uninstall intel deeplearning deployment toolkit'
sudo /opt/intel/deeplearning_deploymenttoolkit/uninstall.sh

echo 'Done'